In [7]:
!pip install git+https://github.com/huggingface/nlp.git
!pip install transformers

  Cloning https://github.com/huggingface/nlp.git to /tmp/pip-req-build-sohqjqe3
  Running command git clone -q https://github.com/huggingface/nlp.git /tmp/pip-req-build-sohqjqe3
  Created wheel for nlp: filename=nlp-0.3.0-cp36-none-any.whl size=116589 sha256=3ab4126200143d584864ab9f175f45127075314b5d860c7a00c97d019c55689d
  Stored in directory: /tmp/pip-ephem-wheel-cache-fea68h05/wheels/a2/98/bd/15f115cb85f1a049a96b915aa14466d3b51a023cd597e08e0d
Successfully built nlp


In [1]:
!rm -rf tapt

In [2]:
!git clone https://github.com/chrisliu298/tapt.git

Cloning into 'tapt'...
remote: Enumerating objects: 348, done.
remote: Counting objects: 100% (348/348), done.
remote: Compressing objects: 100% (248/248), done.
remote: Total 795 (delta 184), reused 249 (delta 89), pack-reused 447
Receiving objects: 100% (795/795), 147.77 MiB | 21.84 MiB/s, done.
Resolving deltas: 100% (393/393), done.
Checking out files: 100% (68/68), done.


In [1]:
import os
os.chdir("/content/tapt/src")

In [ ]:
import pandas as pd
from pprint import pprint
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

from utils.data_pipeline import prepare_data
from utils.data_pipeline import prepare_custom_data
from utils.metrics import compute_metrics


def tokenize(batch):
    """Tokenize a batch of data (with padding and truncation).

    Arg:
        batch: A batch of training data.
    """
    return tokenizer(
        batch["text"], padding="max_length", truncation=True, max_length=512
    )

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
# Load model
model = AutoModelForSequenceClassification.from_pretrained("distilroberta-base")

# Load dataset
train_dataset, val_dataset, test_dataset = prepare_data(
    tokenize_func=tokenize,
    dataset_name="yelp_polarity",
    train_count=10,
    train_size=5,
    val_size=5,
    test_count=10,
    test_size=5,
    non_test_size=5,
    use_all_test=False,
    seed=42,
)
# Load custom data
augmented = prepare_custom_data(
    tokenize_func=tokenize, dataset_name="/content/nlp_yelp_train.tsv"
)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'cl